In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm

import torch
import torch.nn as nn
import torchvision

### defining hyper-parameters ###

In [2]:
input_size = 92       # number of features in each time series step
sequence_length = 2   # this will change with longer trips

hidden_size = 128     # number of hidden nodes in each step
num_layers = 2        # number of layers inside one 'pass'

num_classes = 79      # number of countries seen in threestops
num_epochs = 100      #
batch_size = 16       #
lr = 0.001            # learning rate

### data loading###

In [3]:
# Device configuration (not needed as my laptop contains no GPUs)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set paths
Xtrainpath = './threecountryXtrain.csv'
Ytrainpath = './threecountryYtrain.csv'

# load into pandas dataframe
Xtraindf = pd.read_csv(Xtrainpath)
Ytraindf = pd.read_csv(Ytrainpath)

# drop the picked up index from importing
Xtraindf.drop('Unnamed: 0', axis=1, inplace=True)
Ytraindf.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
# make tensors out of the training data
Xtraintensor = torch.tensor(Xtraindf.values)
Ytraintensor = torch.tensor(Ytraindf.values)

In [5]:
print(Xtraintensor)
print(Ytraintensor)

# remove one-hot-encoding of Ytrain
_, Ytrainvaluetensor = torch.max(Ytraintensor, dim=1)
print(Ytrainvaluetensor.shape)

tensor([[ 9627,     3,     0,  ...,     0,     0,     0],
        [10332,     3,     1,  ...,     0,     0,     0],
        [ 9337,     1,     0,  ...,     0,     0,     0],
        ...,
        [ 2436,     2,     1,  ...,     0,     0,     0],
        [  359,     1,     0,  ...,     0,     0,     0],
        [  359,     1,     0,  ...,     0,     0,     0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
torch.Size([593])


In [6]:
# save a copy of data which has been reshaped so all
# sequence data is stored in one tensor (593 total examples)
Xttgrouped = torch.reshape(Xtraintensor,(593, 92*2))

In [7]:
print(Xttgrouped.shape)
print(Ytraintensor.shape)

torch.Size([593, 184])
torch.Size([593, 79])


In [8]:
# initialize empty array
data_for_loader = []

# fill it
for i in range(0, len(Xttgrouped)):
    data_for_loader.append((Xttgrouped[i], Ytrainvaluetensor[i]))

In [9]:
data_for_loader[0]

(tensor([ 9627,     3,     0,     7,  5581,     1,     0,     0,     0,     0,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0, 10332,     3,     1,     7, 33177,     1,     0,     0,
             0,     0,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

Now that our data looks like it should, we can first split it into train and test sets, then pass these to the DataLoader class to batch over them

### splitting the data into train / test

In [10]:
# specify what proportion of the dataset should be included in training set
train_proportion = 0.8

train_size = int(train_proportion * len(data_for_loader))
test_size = len(data_for_loader) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data_for_loader, [train_size, test_size])

In [11]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = batch_size,
                                          shuffle=False)

In [12]:
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = batch_size,
                                          shuffle=False)

In [13]:
# note: data_loader holds ALL of the data, now batched,
# without a train / test split
data_loader = torch.utils.data.DataLoader(data_for_loader,
                                          batch_size=batch_size,
                                          shuffle=False)

In [14]:
# check out the shape of the 'block' we're passing into our model
for d in train_loader:
    # d[0][0] should be our first length 2 sequence
    print(d[0][0].reshape(sequence_length, input_size))
    
    # d[0][1] should be our first length 2 sequence's last country of the trip
    print(d[1][0])
    break

tensor([[ 2661,     1,     0,     6, 63454,     1,     0,     0,     0,     0,
             0,     0,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  359,     2,     0,     6, 55763,     0,     1,     0,     0,     0,
             0,     0,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     1,     0,     0,     0,     0,
             0,     0,     0

### creating the model object###

In [15]:
# RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # X -> batch_size, sequence_length, input_size
        # X -> -1, 2, 93
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # note: input x(0) is used because we specified batch_first=True
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) # initial hidden state
        
        out, _ = self.rnn(x, h0)
        # out: batch_size, sequence_length, hidden_size
        # out (-1, 2, 128)
        out = out[:, -1, :]
        # out (-1, 128)
        out = self.fc(out)
        return out

In [16]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

### train the model ###

In [17]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()                  # applies the softmax for us
optimizer = torch.optim.Adam(model.parameters(),
                              lr = lr)


In [18]:
n_total_steps = len(train_loader)

In [19]:
# training loop
for epoch in range(num_epochs):
    for i, (Xtrain, Ytrain) in enumerate(train_loader):
        
        # reshape the data as needed
        Xtrain = Xtrain.reshape(-1, sequence_length, input_size).float()
        
        # forward
        outputs = model(Xtrain)
        loss = criterion(outputs, Ytrain)
        
        # backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')       

epoch 1 / 100, step 10/30, loss = 3.3913
epoch 1 / 100, step 20/30, loss = 3.3444
epoch 1 / 100, step 30/30, loss = 3.9164
epoch 2 / 100, step 10/30, loss = 3.3081
epoch 2 / 100, step 20/30, loss = 3.2332
epoch 2 / 100, step 30/30, loss = 3.9551
epoch 3 / 100, step 10/30, loss = 3.2423
epoch 3 / 100, step 20/30, loss = 3.2010
epoch 3 / 100, step 30/30, loss = 3.8223
epoch 4 / 100, step 10/30, loss = 3.1705
epoch 4 / 100, step 20/30, loss = 3.1940
epoch 4 / 100, step 30/30, loss = 3.7260
epoch 5 / 100, step 10/30, loss = 3.1427
epoch 5 / 100, step 20/30, loss = 3.1881
epoch 5 / 100, step 30/30, loss = 3.6487
epoch 6 / 100, step 10/30, loss = 3.1080
epoch 6 / 100, step 20/30, loss = 3.1857
epoch 6 / 100, step 30/30, loss = 3.6032
epoch 7 / 100, step 10/30, loss = 3.0706
epoch 7 / 100, step 20/30, loss = 3.1799
epoch 7 / 100, step 30/30, loss = 3.5331
epoch 8 / 100, step 10/30, loss = 3.0528
epoch 8 / 100, step 20/30, loss = 3.1639
epoch 8 / 100, step 30/30, loss = 3.4923
epoch 9 / 100, s

epoch 67 / 100, step 10/30, loss = 2.4670
epoch 67 / 100, step 20/30, loss = 2.9204
epoch 67 / 100, step 30/30, loss = 3.0643
epoch 68 / 100, step 10/30, loss = 2.4330
epoch 68 / 100, step 20/30, loss = 2.8966
epoch 68 / 100, step 30/30, loss = 3.0441
epoch 69 / 100, step 10/30, loss = 2.4323
epoch 69 / 100, step 20/30, loss = 2.8678
epoch 69 / 100, step 30/30, loss = 3.0053
epoch 70 / 100, step 10/30, loss = 2.4068
epoch 70 / 100, step 20/30, loss = 2.8273
epoch 70 / 100, step 30/30, loss = 3.0213
epoch 71 / 100, step 10/30, loss = 2.5584
epoch 71 / 100, step 20/30, loss = 2.8385
epoch 71 / 100, step 30/30, loss = 3.3524
epoch 72 / 100, step 10/30, loss = 2.5266
epoch 72 / 100, step 20/30, loss = 2.8241
epoch 72 / 100, step 30/30, loss = 3.2772
epoch 73 / 100, step 10/30, loss = 2.5302
epoch 73 / 100, step 20/30, loss = 2.8496
epoch 73 / 100, step 30/30, loss = 3.2847
epoch 74 / 100, step 10/30, loss = 2.5935
epoch 74 / 100, step 20/30, loss = 2.8402
epoch 74 / 100, step 30/30, loss =

### test

In [20]:
# stop computing gradients for memory efficiency
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (Xtest, Ytest) in enumerate(test_loader):
        
        # reshape the data as needed
        Xtest = Xtest.reshape(-1, sequence_length, input_size).float()
        
        outputs = model(Xtest)
        
        # returns value, index
        _, predictions = torch.max(outputs, 1)
        
        n_samples += Xtest.shape[0]
        
        n_correct += (predictions == Ytest).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc}')

accuracy = 15.126050420168067
